In [ ]:
# 📚 필수 라이브러리 설치 및 임포트
import numpy as np
import pandas as pd
import os
import pickle
import joblib
import re
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# GPU 사용 가능 여부 확인
print(f"GPU 사용 가능: {torch.cuda.is_available()}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")

In [ ]:
# ==========================================================
# 📊 데이터 로드 및 기본 정보 확인
# 로컬 경로에서 학습 데이터를 불러옵니다.
train_df = pd.read_csv('train.csv')

print(f"🔍 데이터 형태: {train_df.shape}")
print(f"🎯 타겟 분포: {train_df['rule_violation'].value_counts().to_dict()}")
print("\n📋 데이터 샘플:")
display(train_df.head())

In [ ]:
# ==========================================================
# 🛠️ 특징 엔지니어링 함수 정의
def count_urls(text):
    """URL 개수 세기"""
    return len(re.findall(r'https?://\S+|www\.\S+', str(text)))

def count_exclaims(text):
    """감탄표 개수 세기"""
    return str(text).count('!')

def count_questions(text):
    """물음표 개수 세기"""
    return str(text).count('?')

def upper_ratio(text):
    """대문자 비율 계산"""
    s = str(text)
    letters = [c for c in s if c.isalpha()]
    if not letters:
        return 0.0
    upp = sum(1 for c in letters if c.isupper())
    return upp / len(letters)

def repeat_char_max(text):
    """연속된 문자의 최대 길이"""
    longest = 1
    last = ''
    cur = 0
    for ch in str(text):
        if ch == last:
            cur += 1
        else:
            longest = max(longest, cur)
            cur = 1
            last = ch
    longest = max(longest, cur)
    return longest

def jaccard_similarity(text1, text2):
    """자카드 유사도 계산"""
    set1 = set(str(text1).lower().split())
    set2 = set(str(text2).lower().split())
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union > 0 else 0.0

# ==========================================================
# 🔧 특징 생성 함수
def create_features(df):
    """EDA에서 발견한 유용한 특징들을 생성"""
    df = df.copy()
    
    print("📏 기본 텍스트 특징 생성 중...")
    df['body_len'] = df['body'].astype(str).str.len()
    df['rule_len'] = df['rule'].astype(str).str.len()
    df['body_words'] = df['body'].astype(str).str.split().str.len()
    
    print("🎨 스타일 특징 생성 중...")
    df['url_cnt'] = df['body'].apply(count_urls)
    df['exc_cnt'] = df['body'].apply(count_exclaims)
    df['q_cnt'] = df['body'].apply(count_questions)
    df['upper_rt'] = df['body'].apply(upper_ratio)
    df['rep_run'] = df['body'].apply(repeat_char_max)
    
    print("😊 이모지 및 특수문자 특징 생성 중...")
    emoji_set = set("😀😃😄😁😆😅😂🤣🥲😊🙂🙃😉😍😘😗😙😚😋😛😝😜🤪🤨🧐🤓😎🤩🥳😏😒😞😔😟😕🙁☹️😣😖😫😩😤😠😡🤬")
    df['emoji_rt'] = df['body'].apply(
        lambda s: sum(1 for ch in str(s) if ch in emoji_set) / max(len(str(s)), 1)
    )
    
    print("📚 어휘 기반 특징 생성 중...")
    profanity_words = {'idiot', 'moron', 'stupid', 'dumb', 'retard', 'asshole', 
                      'bastard', 'fuck', 'shit', 'damn', 'bitch'}
    ad_words = {'free', 'win', 'offer', 'discount', 'promo', 'sale', 'subscribe',
                'click', 'visit', 'buy', 'deal', 'coupon', 'limited'}
    
    df['has_profanity'] = df['body'].apply(
        lambda x: int(any(word in str(x).lower() for word in profanity_words))
    )
    df['has_adword'] = df['body'].apply(
        lambda x: int(any(word in str(x).lower() for word in ad_words))
    )
    
    print("🔗 규칙-댓글 상호작용 특징 생성 중...")
    df['rule_body_jaccard'] = [
        jaccard_similarity(rule, body) 
        for rule, body in zip(df['rule'], df['body'])
    ]
    
    print("✅ 특징 생성 완료!")
    return df

# ==========================================================
# 🤖 Cross-Encoder 입력 준비 함수
def prepare_cross_encoder_input(rule, body, positive_ex1=None, positive_ex2=None, 
                               negative_ex1=None, negative_ex2=None):
    """Cross-Encoder를 위한 입력 텍스트 준비"""
    rule_text = str(rule).strip()
    comment_text = str(body).strip()
    
    examples_text = ""
    if pd.notna(positive_ex1) and str(positive_ex1).strip():
        examples_text += f" [긍정예시] {str(positive_ex1).strip()}"
    if pd.notna(negative_ex1) and str(negative_ex1).strip():
        examples_text += f" [부정예시] {str(negative_ex1).strip()}"
    
    full_input = f"[규칙] {rule_text}{examples_text} [댓글] {comment_text}"
    return full_input

# ==========================================================
# 📊 데이터 전처리 및 특징 생성
print("🔧 데이터 전처리 시작...")

# 특징 생성 실행
train_df = create_features(train_df)

In [ ]:
# 수치 특징 정의
numerical_cols = [
    'body_len', 'rule_len', 'body_words', 'url_cnt', 'exc_cnt', 'q_cnt',
    'upper_rt', 'rep_run', 'emoji_rt', 'has_profanity', 'has_adword', 
    'rule_body_jaccard'
]

# 수치 특징 정규화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numerical_features = scaler.fit_transform(train_df[numerical_cols])

# Cross-Encoder 입력 데이터 준비
print("🔄 Cross-Encoder 입력 데이터 준비 중...")
ce_inputs = []
labels = []

for idx, row in tqdm(train_df.iterrows(), total=len(train_df), desc="입력 데이터 처리"):
    ce_input = prepare_cross_encoder_input(
        row['rule'], 
        row['body'],
        row.get('positive_example_1'),
        row.get('positive_example_2'),
        row.get('negative_example_1'),
        row.get('negative_example_2')
    )
    ce_inputs.append(ce_input)
    labels.append(int(row['rule_violation']))

ce_inputs = np.array(ce_inputs)
labels = np.array(labels)

print(f"✅ {len(ce_inputs)}개의 Cross-Encoder 입력 쌍 준비 완료")

In [ ]:
# ==========================================================
# 🏗️ 최종 모델 훈련 (전체 데이터 사용)
from sentence_transformers import CrossEncoder, InputExample

print("🏗️ 전체 데이터셋으로 최종 모델 훈련 시작!")

# 모델 저장 경로 설정
output_model_path = './model_output/final_cross_encoder_model'
os.makedirs(os.path.dirname(output_model_path), exist_ok=True) # 저장 폴더 생성

# 최종 모델 초기화
model_name = 'microsoft/deberta-v3-small'
final_model = CrossEncoder(model_name, num_labels=1, device=device)

# 전체 훈련 데이터로 예시 생성
print("📚 전체 훈련 예시 생성 중...")
train_examples = []

for i in tqdm(range(len(ce_inputs)), desc="최종 훈련 데이터 처리"):
    ce_input = ce_inputs[i]
    
    if '[댓글]' in ce_input:
        rule_part = ce_input.split('[댓글]')[0].strip()
        comment_part = ce_input.split('[댓글]')[1].strip()
    else:
        parts = ce_input.split()
        mid = len(parts) // 2
        rule_part = ' '.join(parts[:mid])
        comment_part = ' '.join(parts[mid:])
    
    train_examples.append(
        InputExample(texts=[rule_part, comment_part], label=float(labels[i]))
    )

# 최종 훈련 설정
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
warmup_steps = max(1, int(len(train_dataloader) * 0.1))

print(f"🚀 최종 모델 훈련 (에폭: 4, 배치: 16)")

# 최종 모델 훈련
final_model.fit(
    train_dataloader=train_dataloader,
    epochs=4,
    warmup_steps=warmup_steps,
    output_path=output_model_path,
    save_best_model=True,
    show_progress_bar=True
)

In [ ]:
# ==========================================================
# 💾 모델 및 전처리 객체 저장
print("💾 모델 및 전처리 객체 저장 중...")

# 모델 저장 경로 변수 사용
output_dir = './model_output'

# 1. Cross-Encoder 모델은 이미 output_path에 저장됨
print(f"✅ Cross-Encoder 모델 저장 완료: {os.path.abspath(output_dir)}/final_cross_encoder_model")

# 2. Scaler 저장
joblib.dump(scaler, os.path.join(output_dir, 'scaler.pkl'))
print(f"✅ Scaler 저장 완료: {os.path.join(output_dir, 'scaler.pkl')}")

# 3. 수치 특징 컬럼명 저장
with open(os.path.join(output_dir, 'numerical_cols.pkl'), 'wb') as f:
    pickle.dump(numerical_cols, f)
print(f"✅ 수치 특징 컬럼명 저장 완료: {os.path.join(output_dir, 'numerical_cols.pkl')}")

# 4. 특징 엔지니어링 함수들을 포함한 유틸리티 저장
feature_utils = {
    'count_urls': count_urls,
    'count_exclaims': count_exclaims,
    'count_questions': count_questions,
    'upper_ratio': upper_ratio,
    'repeat_char_max': repeat_char_max,
    'jaccard_similarity': jaccard_similarity,
    'create_features': create_features,
    'prepare_cross_encoder_input': prepare_cross_encoder_input
}

with open(os.path.join(output_dir, 'feature_utils.pkl'), 'wb') as f:
    pickle.dump(feature_utils, f)
print(f"✅ 특징 엔지니어링 함수들 저장 완료: {os.path.join(output_dir, 'feature_utils.pkl')}")

In [ ]:
# ==========================================================
# 📋 저장된 파일들 확인
output_dir = './model_output'
print(f"\n📋 '{output_dir}' 폴더에 저장된 파일들:")
for file in os.listdir(output_dir):
    file_path = os.path.join(output_dir, file)
    if os.path.isfile(file_path):
        file_size = os.path.getsize(file_path) / (1024*1024)  # MB
        print(f"  - {file}: {file_size:.2f} MB")

print(f"\n🎉 모델 학습 및 저장 완료!")
print(f"📦 이제 '{output_dir}' 폴더에 저장된 파일들을 사용하여 추론을 진행할 수 있습니다.")

# 저장된 모델로 빠른 테스트
print("\n🧪 저장된 모델 빠른 테스트...")
test_rule = "No spam or promotional content"
test_body = "Check out this amazing deal! Buy now!"

test_input = prepare_cross_encoder_input(test_rule, test_body)
if '[댓글]' in test_input:
    rule_part = test_input.split('[댓글]')[0].strip()
    comment_part = test_input.split('[댓글]')[1].strip()
else:
    parts = test_input.split()
    mid = len(parts) // 2
    rule_part = ' '.join(parts[:mid])
    comment_part = ' '.join(parts[mid:])

# 훈련이 끝난 final_model 변수를 사용하여 바로 예측
test_pred = final_model.predict([[rule_part, comment_part]])
test_prob = torch.sigmoid(torch.tensor(test_pred)).item()

print(f"테스트 규칙: {test_rule}")
print(f"테스트 댓글: {test_body}")
print(f"위반 확률: {test_prob:.4f}")

print("\n✅ 모든 작업 완료!")